Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [ ]:
from random import choices
from dataclasses import dataclass
from random import random, randint, choice, gauss, shuffle
from copy import copy
import numpy as np
import math
import matplotlib.pyplot as plt 
import lab9_lib

In [ ]:
POPULATION_SIZE = 100
#OFFSPRING_SIZE = 70
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = 0.10
GENOME_LENGTH = 1000
STOP = 4500

## INDVIDUAL STRUCTURE

In [ ]:
@dataclass
class Individual:
    fitness: float
    genotype: list[int]

    def __str__(self):
        return f"fitness: {self.fitness:.2%} - genotype: {''.join(str(int(g)) for g in self.genotype)}"
  
    

## SELECTION

In [ ]:
def select_parent( pop):  
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]  
    champion = max(pool, key=lambda i: i.fitness)  
   
    return champion

def roulette_wheel_selection(pop):
    
    total_fitness = sum(i.fitness for i in pop)
    probabilities = [i.fitness / total_fitness for i in pop]
        
    selected_index = choices(range(len(pop)), weights=probabilities)[0]
    return pop[selected_index]

## MUTATION AND CROSSOVER OPERATORS

In [ ]:

def mutation(ind: Individual) -> Individual:
    offspring = copy(ind)
    # lets mutate one bit in the genome
    pos = randint(0, GENOME_LENGTH - 1)
    offspring.genotype[pos] = not offspring.genotype[pos]
    offspring.fitness = None
    return offspring


def swap_mutation(ind: Individual) -> Individual:
    offspring = copy(ind)
    #lets swap two random bit in the genome
    pos1 = 0
    pos2 = 0
    while pos1 == pos2:
        pos1 = randint(0, GENOME_LENGTH - 1)
        pos2 = randint(0, GENOME_LENGTH - 1)
        if pos1 != pos2:
            break
    
    temp = offspring.genotype[pos1] 
    offspring.genotype[pos1] = offspring.genotype[pos2]
    offspring.genotype[pos2] = temp

    offspring.fitness = None
    return offspring

def gaussian_mutation(ind: Individual) -> Individual:
    offspring = copy(ind)
    σ = 0.1

    for i in range(len(offspring.genotype)):
       
        variation = gauss(0, σ)
        offspring.genotype[i] = 1 if variation > 0 else 0

    offspring.fitness = None  
    return offspring


def reverse_mutation(ind: Individual) -> Individual:
    offspring = copy(ind)

    pos1 = 0
    pos2 = 0
    while pos1 == pos2:
        pos1 = randint(0, GENOME_LENGTH - 1)
        pos2 = randint(0, GENOME_LENGTH - 1)
        if pos1 != pos2:
            break

    if pos1 > pos2:
        temp = pos1
        pos1 = pos2
        pos2 = temp

    offspring.genotype[pos1:pos2+1] = reversed(offspring.genotype[pos1:pos2+1])
    
    offspring.fitness = None  
    return offspring

def random_resetting(ind: Individual) -> Individual:
    offspring = copy(ind)

    while True:
        pos1 = randint(0, GENOME_LENGTH - 1)
        pos2 = randint(0, GENOME_LENGTH - 1)
        if pos1 != pos2:
            break

    offspring.genotype[pos1] = choice([0,1])
    offspring.genotype[pos2] = choice([0,1])

      
    offspring.fitness = None  
    return offspring

def scramble_mutation(ind: Individual) -> Individual:
    offspring = copy(ind)

    pos1 = 0
    pos2 = 0
    while pos1 == pos2:
        pos1 = randint(0, GENOME_LENGTH - 1)
        pos2 = randint(0, GENOME_LENGTH - 1)
        if pos1 != pos2:
            break

    if pos1 > pos2:
        temp = pos1
        pos1 = pos2
        pos2 = temp

    genes_to_scramble = list(offspring.genotype[pos1:pos2+1])
    shuffle(genes_to_scramble)
    offspring.genotype[pos1:pos2+1] = genes_to_scramble
 
   
    assert len(offspring.genotype) == GENOME_LENGTH

    offspring.fitness = None  
    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    assert len(ind1.genotype) == len(ind2.genotype)
    # we need to create a new individual
    cut_point = randint(0, GENOME_LENGTH - 1)
    offspring = Individual(
        fitness=None,
        genotype=ind1.genotype[:cut_point]
        + ind2.genotype[cut_point:],
    )

    assert len(offspring.genotype) == GENOME_LENGTH
    return offspring

def two_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    
    while True:
        cut_point_one = randint(0, GENOME_LENGTH -1)
        cut_point_two = randint(0, GENOME_LENGTH -1)
        if cut_point_one != cut_point_two:
            break
    

    if cut_point_one > cut_point_two:
        temp = cut_point_one
        cut_point_one = cut_point_two
        cut_point_two = temp
    
    offspring1 = Individual(
        fitness=None,
        genotype = ind1.genotype[:cut_point_one]
        + ind2.genotype[cut_point_one:cut_point_two]
        + ind1.genotype[cut_point_two:]
    )

    return offspring1



def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
   
    offspring = copy(ind1)
    for i in range(len(offspring.genotype)):
        if choice([True, False]):
            offspring.genotype[i] = ind2.genotype[i]

    assert len(offspring.genotype) == GENOME_LENGTH
    return offspring    


def n_point_crossover(ind1: Individual, ind2: Individual) -> Individual:
    num_of_cut_points = randint(4, 7)

    cut_points = []
    
    while len(cut_points) != num_of_cut_points:
        already_in = True
        if len(cut_points) == 0:
            cut_point = randint(0, 50)
        else:
            while already_in:
                cut_point = randint(0, GENOME_LENGTH - 1)
                if cut_point not in cut_points:
                    already_in = False


        cut_points.append(cut_point)

    cut_points = sorted(cut_points, reverse=False)
    offspring_genotype = []
    parent_switch = False
    last_cut = 0
    for cut_point in cut_points:
        end_point = GENOME_LENGTH if cut_point == cut_points[-1] else cut_point

        if parent_switch:
            offspring_genotype += ind2.genotype[last_cut:end_point]
        else:
            offspring_genotype += ind1.genotype[last_cut:end_point]

        last_cut = cut_point
        parent_switch = not parent_switch

    offspring = Individual(
        fitness=None,
        genotype = offspring_genotype
    )

    assert len(offspring.genotype) == GENOME_LENGTH

    return offspring
    

## GENERATION OF THE PROBLEM AND INITIAL POPULATION

In [ ]:
PROBLEM_INSTANCE = 1
fitness = lab9_lib.make_problem(PROBLEM_INSTANCE)

In [ ]:

population = []
for n in range(POPULATION_SIZE):
    ind = choices([0, 1], k=GENOME_LENGTH)
    population.append(Individual(fitness=fitness(ind), genotype= ind))

for ind in sorted(population, key=lambda i: i.fitness, reverse=True):
    print(ind)
   

## Generational GA + Elitism

In [ ]:
best_percentage = 10

generations = 0

stop_condition = STOP
old_best = 0
increment_rate = 0.01
changed = False
mut = [mutation, swap_mutation, reverse_mutation, scramble_mutation]
crossover = [two_cut_xover, one_cut_xover]

x_axis = []
y_axis = []

while population[0].fitness < 1.00 and generations <= 12_000:
    generations +=1
    offspring = list()
    
    sorted_population = sorted(population, key=lambda i: i.fitness, reverse=True) #i'm putting the  higher fitness values on top and then i selected the top individual
    elite_size = int((best_percentage * len(population))/100)
    elite = sorted_population[:elite_size]
    offspring.extend(elite)

    while len(offspring) < POPULATION_SIZE:

        if MUTATION_PROBABILITY > 15:
            type = 3
        else:
            if len(offspring) <= POPULATION_SIZE - 15:
                type = choice([0,3])
            elif len(offspring) > POPULATION_SIZE - 15 and len(offspring) <= POPULATION_SIZE -5:
                type = choice([1,2])
            else:
                type = 1

        if random() < MUTATION_PROBABILITY:  
            p = select_parent(population)
            o = mut[type](p)
            o.fitness = fitness(o.genotype)
        else:
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = choice(crossover)(p1, p2)
            o.fitness = fitness(o.genotype)
        offspring.append(o)
        

    assert len(offspring) == POPULATION_SIZE
    population = copy(offspring)
    assert len(population) == POPULATION_SIZE


    print(population[0]) #for each generation i print the best fittest individual  of the population
    x_axis.append(generations)
    y_axis.append(population[0].fitness)

  
    if(population[0].fitness == old_best):
        stop_condition-=1
        if stop_condition == STOP - 50:
            MUTATION_PROBABILITY += increment_rate
            MUTATION_PROBABILITY = min(0.3, MUTATION_PROBABILITY)
            stop_condition = STOP
            old_best = population[0].fitness
            changed = True
            print('mutation_rate: ', MUTATION_PROBABILITY)
        if(stop_condition == 0):
            break
    else:
        if changed:
            MUTATION_PROBABILITY -= 0.08
            MUTATION_PROBABILITY = max(0.10, MUTATION_PROBABILITY)
        stop_condition = STOP
        old_best = population[0].fitness
        
print(f'problem solved in {generations:,} generations  - fitness call: {fitness.calls}')


## Fitness History

In [ ]:

plt.plot(x_axis, y_axis)

plt.xlabel('Generations')
plt.ylabel('Fitness')


plt.title(f'Fitness over generations - Problem istance {PROBLEM_INSTANCE}')

plt.show()